# Fudan PRML 23Spring Assignment2


<font color='red'>*Your Name, Student ID and Date: [Name], [Student ID], [Date]*</font>

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission.*


<font color='red' size = 3> *DDL: 2023.06.11 23:59* </font>

## 1. FDUNN: your toy torch-like deep learning library (60 points)

In this assignment, you will fist implement your own torch-like deep learning library with `numpy`, named `fdunn`.

PyTorch: [Link](https://pytorch.org/)

In [2]:
# setup code, auto reload your .py file
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# numpy
import numpy as np
np.random.seed(233)

In [4]:
# matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

You have to impletement several standard deep neural modelwork modules in the `./fdunn` folder:
1.   linear/conv/pooling
2.   activation
3.   loss
4.   optim
5.   trainer

We have written most of the code for you already, and you only need to fill in the most essential parts. We have also prepared several test cases for you to check if your code works correctly.

Furthermore, you can also test the accuracy of your code by comparing its output with the output of sk-learn.

In [5]:
import sys
sys.path.append(r'C:\Users\Life_Dancer\Desktop\PRML-Spring23-FDU\assignment2\fdunn')

from fdunn import *

In [6]:
from fdunn.modules.linear import Linear
from fdunn.modules.activation import Sigmoid
from fdunn.modules.loss import BCELoss, CrossEntropyLoss
from fdunn.optim.sgd import SGD

class FNN:
    def __init__(self, in_features, hidden_size, num_classes):
        self.linear1 = Linear(in_features, hidden_size)
        self.sigmoid = Sigmoid()
        self.linear2 = Linear(hidden_size, num_classes)
    
    def forward(self, input):
        output = self.linear1.forward(input)
        output = self.sigmoid.forward(output)
        output = self.linear2.forward(output)
        return output
    
    def backward(self, output_grad):
        output_grad = self.linear2.backward(output_grad)
        output_grad = self.sigmoid.backward(output_grad)
        output_grad = self.linear1.backward(output_grad)
        return output_grad

In [7]:
model = FNN(784, 128, 10)
criterion = CrossEntropyLoss(model)

lr = 0.01
optimizer = SGD(model, lr)

In [8]:
from fdunn.data.data_loader import get_dataset

dataset = 'MNIST'
trainloader, testloader = get_dataset('MNIST')

In [9]:
def epoch(mode, dataloader, model, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0, 0, 0

    for i_batch, datum in enumerate(dataloader):
        img = datum[0]
        lab = datum[1]

        n_b = lab.shape[0]

        output = model.forward(img)
        # print(output)
        loss = criterion.forward(output, lab)

        predicted = np.argmax(output, 1)
        correct = (predicted == lab).sum()

        loss_avg += loss.item()*n_b
        acc_avg += correct.item()
        num_exp += n_b

        if mode == 'train':
            # optimizer.zero_grad() should we implement zero_grad()?
            loss.backward()
            optimizer.step()

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

In [10]:
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "assignment2.ipynb"

In [11]:
import wandb

# wandb.init(sync_tensorboard=False,
#         entity='tongchen',
#         project="PRML",
#         name='{}-{}-{}-{}'.format(dataset, model, lr, criterion)
#     )

train_epochs = 20
lr_schedule = [10, 15]
decay = True

for e in range(train_epochs):

    train_loss, train_acc = epoch("train", dataloader=trainloader, model=model, optimizer=optimizer, criterion=criterion)

    test_loss, test_acc = epoch("test", dataloader=testloader, model=model, optimizer=None, criterion=criterion)

    print("Epoch: {}\tReal Acc: {}\tTest Acc: {}".format(e, train_acc, test_acc))

    # wandb.log({'Train Loss': train_loss}, step=e)
    # wandb.log({'Train Acc': train_acc}, step=e)
    # wandb.log({'Test Loss': test_loss}, step=e)
    # wandb.log({'Test Acc': test_acc}, step=e)
    # wandb.log({'lr': lr}, step=e)

    if e in lr_schedule and decay:
        lr *= 0.1
        optimizer = SGD(model, lr=lr)
        # optimizer.zero_grad()   should we implement zero_grad()?

# wandb.finish()

torch.Size([8, 1, 28, 28])
784
torch.Size([8, 784])
(128, 784)


: 

: 